#### Install the necessary package.

In [1]:
!pip install watson-developer-cloud==1.5

     |████████████████████████████████| 225kB 11.8MB/s eta 0:00:01
     |████████████████████████████████| 716kB 20.3MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 33.7MB/s eta 0:00:01
     |████████████████████████████████| 2.3MB 51.1MB/s eta 0:00:01
     |████████████████████████████████| 174kB 54.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 39.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 42.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 917kB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/81/cd/de/4e0916f623c2d125502e493394fd333ed693960264d4b7e524
  Stored in directory: /home/dsxuser/.cache/pip/wheels/19/4c/a1/0bef832134076abab3461815e92cf41f2518dffe1a5337203b
Successfully built watson-developer-cloud Twisted
  Found existing installation: cryptography 2.5
    Uninstalling cryptography-2.5:
      Successfully uninstalled cryptography-2.5


#### Import the necessary libraries

In [2]:
import requests
import urllib.request
import time
import datetime
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd 

#### I've pre-defined some redundant functions here so as to reduce the complexity and size of the code

In [3]:
#function to check if we have access to the website
def access_to_server(a):
    if(access.status_code == 200):
        print("Access to PlaneCrashInfo.com: Granted")
    else:
        print("Access to PlaneCrashInfo.com: Denied")
        
#function to pause for a little so that the website server does not flag us as a spammer
def take_a_break():
    time.sleep(0.1)      #we pause for 3 seconds

#function to access the website
def access_website(url):
    access = None
    access = requests.get(url,timeout=10)
    return access   

def get_data(search_string, regex):
    pattern = re.compile(regex)
    matches = pattern.finditer(search_string)

    for match in matches:
        match = match.span()

    span = search_string[match[0]:match[1]]
    return span

#### Next, we set the url to the Plane Crash Info website and access the site with our requests library. We also set the base url to create a dictionary with years as the key.

In [4]:
base_url = "http://www.planecrashinfo.com"
access = requests.get('http://www.planecrashinfo.com/database.htm')
access_to_server(access)
source = BeautifulSoup(access.text, 'html.parser')

Access to PlaneCrashInfo.com: Granted


 #### Here we build a dictionary with the url stores as the value and the year as the key

In [82]:
a_tags = source.find_all("a")
years = {a.text.strip(): {"url": base_url + a["href"] if a["href"][0] == "/" else base_url + "/" + a["href"]} for a in a_tags if a.text.strip().isdigit()}

#### Here we pause for a little to avoid getting flagged as a spammer.

In [83]:
take_a_break()

In [102]:
done = 0
for year, database in years.items():       #first we loop for each year from 1920 until 2019
    if year == '1921':
        break
    take_a_break()
    response_year = access_website(database["url"])
    access_to_server(response_year)
    parser_year = BeautifulSoup(response_year.content, 'html.parser')
    a_tags2 = parser_year.find_all("a")
    a_tags2 = [a for a in a_tags2 if "Return to Home" not in a.text]
    for a in a_tags2:
            # request to crash detail page
            response_crash = access_website(base_url + "/" + year + a["href"] if a["href"][0] == "/" else base_url + "/" + year + "/" + a["href"])
            parser_crash = BeautifulSoup(response_crash.content, 'html.parser')
            tr_tags = parser_crash.find_all("tr")
            tr_tags = tr_tags[1:]

            database = [tr.find_all("td")[1].text.strip() for tr in tr_tags]
    if year == '1920':
        break      
    database

Access to PlaneCrashInfo.com: Granted


In [103]:
x = database
x

['December 24, 1920',
 '?',
 'Barcelona, Spain',
 'Aeropostale',
 '?',
 '?',
 'Breguet 14',
 'F-ALBO',
 '57',
 '1 \xa0 (passengers:0\xa0 crew:1)',
 '1 \xa0 (passengers:0\xa0 crew:1)',
 '0',
 'The aircraft stalled and crashed when an engine failed.']

In [107]:
import unicodedata
#Splitting the data in the aboard row
aboard_row = x[9]
#aboard_row = aboard_row.replace(u'\xa0', u' ')
aboard_row = unicodedata.normalize("NFKD", aboard_row)
aboard_row = re.sub(' +', ' ',aboard_row)
split_aboard_row = aboard_row.split(" ")
#find the total number of people aboard
total_aboard = split_aboard_row[0]
#find the total number of passengers aboard
passengers_aboard = split_aboard_row[2]


In [109]:
split_aboard_row

['1', '(passengers:0', 'crew:1)']

In [70]:
#Splitting the data in the aboard row
aboard_row = x[9]
split_aboard_row = aboard_row.split(" ")
#find the total number of people aboard
total_aboard = split_aboard_row[0]
#find the total number of passengers aboard
passengers_aboard = split_aboard_row[2]
passengers_aboard = re.findall(r'\d+', passengers_aboard)
passengers_aboard = passengers_aboard[0]
#find the total number of crew members aboard
crew_aboard = split_aboard_row[3]
crew_aboard = re.findall(r'\d+', crew_aboard)
crew_aboard = crew_aboard[0]

#Splitting the data in the fatalities row
fatalities_row = x[10]
split_fatalities_row = fatalities_row.split(" ")
#find the total number of fatalities
total_fatalities = split_fatalities_row[0]
#find the total number of passenger fatalities
passenger_fatalities = split_fatalities_row[2]
passenger_fatalities = re.findall(r'\d+', passenger_fatalities)
passenger_fatalities = passenger_fatalities[0]
#find the total number of crew fatalities
crew_fatalities = split_fatalities_row[3]
crew_fatalities = re.findall(r'\d+', crew_fatalities)
crew_fatalities = crew_fatalities[0]

#save data for reorganization
ground = x[11]
summary = x[12]

In [71]:
x[9] = total_aboard
x[10] = passengers_aboard
x[11] = crew_aboard
x[12] = total_fatalities

u= [passenger_fatalities, crew_fatalities, ground, summary]
x.extend(u)
x


['September 17, 1908',
 '1718',
 'Fort Myer, Virginia',
 'Military - U.S. Army',
 '?',
 'Demonstration',
 'Wright Flyer III',
 '?',
 '1',
 '2',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 'During a demonstration flight, a U.S. Army flyer flown by Orville Wright nose-dived into the ground from a height of approximately 75 feet, killing Lt. Thomas E. Selfridge, 26, who was a passenger. This was the first recorded airplane fatality in history.  One of two propellers separated in flight, tearing loose the wires bracing the rudder and causing the loss of control of the aircraft.  Orville Wright suffered broken ribs, pelvis and a leg.  Selfridge suffered a crushed skull and died a short time later.']

In [72]:
df = pd.DataFrame(x) 
q= df.T
q

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."


#### Now we start collecting the unstructured data

x = database
            #Splitting the data in the aboard row
            aboard_row = x[9]
            split_aboard_row = aboard_row.split(" ")
            #find the total number of people aboard
            total_aboard = split_aboard_row[0]
            #find the total number of passengers aboard
            passengers_aboard = split_aboard_row[2]
            passengers_aboard = re.findall(r'\d+', passengers_aboard)
            passengers_aboard = passengers_aboard[0]
            #find the total number of crew members aboard
            crew_aboard = split_aboard_row[3]
            crew_aboard = re.findall(r'\d+', crew_aboard)
            crew_aboard = crew_aboard[0]

            #Splitting the data in the fatalities row
            fatalities_row = x[10]
            split_fatalities_row = fatalities_row.split(" ")
            #find the total number of fatalities
            total_fatalities = split_fatalities_row[0]
            #find the total number of passenger fatalities
            passenger_fatalities = split_fatalities_row[2]
            passenger_fatalities = re.findall(r'\d+', passenger_fatalities)
            passenger_fatalities = passenger_fatalities[0]
            #find the total number of crew fatalities
            crew_fatalities = split_fatalities_row[3]
            crew_fatalities = re.findall(r'\d+', crew_fatalities)
            crew_fatalities = crew_fatalities[0]

            #save data for reorganization
            ground = x[11]
            summary = x[12]
            x[9] = total_aboard
            x[10] = passengers_aboard
            x[11] = crew_aboard
            x[12] = total_fatalities

            u= [passenger_fatalities, crew_fatalities, ground, summary]
            x.extend(u)
            df = pd.DataFrame(x) 
            q= df.T
            

done = 0
for year, database in years.items():       #first we loop for each year from 1920 until 2019
    print(year)
    if year == 1969:
        break
    take_a_break()
    response_year = access_website(database["url"])
    access_to_server(response_year)
    parser_year = BeautifulSoup(response_year.content, 'html.parser')
#    if done == 0:
#        break
    a_tags2 = parser_year.find_all("a")
    a_tags2 = [a for a in a_tags2 if "Return to Home" not in a.text]
    for a in a_tags2:

            # request to crash detail page
            response_crash = access_website(base_url + "/" + year + a["href"] if a["href"][0] == "/" else base_url + "/" + year + "/" + a["href"])
            parser_crash = BeautifulSoup(response_crash.content, 'html.parser')

            # get all table content except the first row(table title)
            tr_tags = parser_crash.find_all("tr")
            tr_tags = tr_tags[1:]

            database = [tr.find_all("td")[1].text.strip() for tr in tr_tags]
            # get all aboard
            aboard = database[9]
            all_aboard = get_data(aboard, r'^\d+|^\W')

            # get all passengers aboard
            passengers_aboard = get_data(aboard, r'(?<=\(passengers:)\d+|(?<=\(passengers:)\W')

            # get all crew aboard
            crew_aboard = get_data(
                aboard, r'(?<=crew:)\d+|(?<=crew:)\W')

            # get all fatalities
            fatalities = database[10]

            all_fatalities = get_data(fatalities, r'^\d+|^\W')

            # get all passengers fatalities
            passenger_fatalities = get_data(fatalities, r'(?<=\(passengers:)\d+|(?<=\(passengers:)\W')

            # get all crew fatalities
            crew_fatalities = get_data(fatalities, r'(?<=crew:)\d+|(?<=crew:)\W')

            # remove aboard info and store each value separately
            database.pop(9)
            database.insert(9, all_aboard)
            database.insert(10, passengers_aboard)
            database.insert(11, crew_aboard)

            # remove fatalities info and store each value separately
            database.pop(12)
            database.insert(12, all_fatalities)
            database.insert(13, passenger_fatalities)
            database.insert(14, crew_fatalities)
            take_a_break()